# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Ang Mink Chen
###### Python version: Python 3
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
import numpy as np
import random
import math
import copy
import pandas as pd # Just to tabulate the confusion matrix so that it's easier to be printed

In [2]:
# This function opens a data file in csv, and transforms it into a usable format (2D array)
# Assumptions for this implementation: 
# 1. Data in the datasets provided are always valid: 
#    a) All instances have same amount of attributes. 
#    b) Data are recorded properly (no human errors/typos).
#    c) Missing values are due to exterior factors beyond the scope of this project.

def preprocess(filename):
    f = open(filename, 'r')
    data = []
    target = []
    for line in f.readlines():
        instance = line.strip().split(',')
        data.append(instance[:-1])
        target.append(instance[-1])
    f.close()    
    return data, target


In [3]:
# This function builds a supervised NB model
def train(data, target):
    
    model = {}
    
    # Iterate through each instance
    for ctr in range(len(data)):
        
        ### Count labels occurrences ### 
        label = target[ctr]
        # Build up the dict structure procedurally for labels
        if label not in model.keys(): model[label] = {}
        # Initialize the count to 1 if it has not been encountered before or increment the count by 1
        model[label]['count'] = [lambda:1, lambda:model[label]['count']+1][bool(model[label])]()
        
        ### Count attributes occurrences ###
        attributes = data[ctr]
        # Iterate through each data attribute
        for i in range(len(attributes)):
            attr = attributes[i]
            # Build up the dict structure procedurally for attributes 
            # Attributes are named after their indexes in the dataset (e.g. attribute at index 0 is named 0)
            if i not in model[label].keys(): model[label][i] = {}
            # Initialize the count to 1 if it has not been encountered before or increment the count by 1
            model[label][i][attr] = [lambda:1, lambda:model[label][i][attr]+1][attr in model[label][i].keys()]()
    
    
    ### Make sure each attribute contains all possible attribute values ###
    # Iterate through the model created to find all possible values for each attribute
    attr_values_dict = {}    
    for label in model.keys():
        for attr in model[label].keys():
            if attr == 'count': continue
            if attr not in attr_values_dict.keys(): 
                attr_values_dict[attr] = []
            for attr_val in model[label][attr].keys():
                if attr_val not in attr_values_dict[attr]: 
                    attr_values_dict[attr].append(attr_val)
    
    # Iterate through the model, check if all values of each attribute has been recorded
    # (especially for a value with count 0 as it will not be recorded in the model initially), 
    # and initialize all unrecorded attributes values with count 0
    for label in model.keys():
        for attr in model[label].keys():
            if attr == 'count': continue
            for attr_val in attr_values_dict[attr]:
                if attr_val not in model[label][attr].keys():
                    model[label][attr][attr_val] = 0
                    
            
    return model

In [4]:
# This function predicts the class for an instance or a set of instances based on a trained NB model 
def predict(nb_model, instances, k=1):
    
    # Array to store all the predictions
    predictions = []
        
    for instance in instances:
        
        # Dict structure to store all the class probabilities calculated
        p = {}
        
        ### Get total count of the instances ###
        total_count = 0
        for label in nb_model.keys():
            total_count += nb_model[label]['count']

        ### Calculate the probability for each label prediction ###
        # Iterate through each label
        for label in nb_model.keys():
            
            posterior_prob = 1
            # Iterate through each attribute to calulate posterior probability
            for i in range(len(instance)):
                attr = instance[i]
                # Ignore missing values
                if attr == '?': continue
                # Calculate the probability based on counts recorded in the model    
                posterior_prob *= nb_model[label][i][attr]/nb_model[label]['count']
            
            # Recalculate posterior probability with add-k smoothing if there is a previously unseen test 
            # instance that caused the posterior probability to be 0
            if posterior_prob == 0:
                posterior_prob = 1
                # Iterate through each attribute to calulate posterior probability 
                for i in range(len(instance)):
                    attr = instance[i]
                    # Ignore missing values
                    if attr == '?': continue
                    # Calculate the probability with add-k smoothing  
                    posterior_prob *= (nb_model[label][i][attr]+k)/(nb_model[label]['count']+(len(nb_model[label][i].keys())*k))
            
            # Calculate the prior probability    
            prior_prob = nb_model[label]['count']/(total_count+0.0)
            
            # Calculate the label probability using prior and posterior probability, then store it 
            p[label] = prior_prob * posterior_prob

        ### Get the predicted label with highest probability ###
        prediction = None
        highest_prob = -1
        for label, prob in p.items():
            if prob > highest_prob:
                highest_prob = prob
                prediction = label
        
        # Store the predicted label
        predictions.append(prediction)        
                
    return predictions


In [5]:
# This function evaluates a set of predictions, in a supervised context 
def evaluate(predictions, target):
    
    # Get all the labels
    labels = []
    for t in target:
        if t not in labels:
            labels.append(t)
    
    # Initialize confusion matrix as dict with the required evaluation metrics
    con_matrix = {}
    con_matrix['Precision'] = []
    con_matrix['Recall']    = []
    con_matrix['Accuracy']  = []
    con_matrix['F1_Score']  = []
    
    # Iterate through each class to calculate the classifier's performance (evaluation metrics) 
    # with respect to the class
    for label in labels:
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        # Iterate through all test instances
        for i in range(len(predictions)):
            pred = predictions[i]
            
            # Set the selected label as positive (interested class)
            if pred == label:
                # Predict positive class and the target is positive class
                if pred == target[i]:
                    tp += 1
                # Predict positive class but the target is negative class
                else:   
                    fp += 1
                    
            # Set all other labels as negative (uninterested class)     
            else:
                # Predict negative class but the target is positive class
                if target[i] == label:
                    fn += 1
                # Predict negative class and the target is negative class    
                else:
                    tn += 1
        
        # Perform calculations and store the results for all required evaluation metrics
        precision = [lambda: tp / (tp + fp), lambda: 0][tp + fp == 0]()
        recall    = [lambda: tp / (tp + fn), lambda: 0][tp + fn == 0]()
        accuracy  = [lambda: (tp + tn) / (tp + fp + tn + fn), lambda: 0][tp + fp + tn + fn == 0]()
        f1_score  = [lambda: 2 * (precision * recall) / (precision + recall), lambda: 0][(precision + recall) == 0]()
        
        con_matrix['Precision'].append(precision)
        con_matrix['Recall'].append(recall)
        con_matrix['Accuracy'].append(accuracy)
        con_matrix['F1_Score'].append(f1_score)
    
    ### Create DataFrame to display the confusion matrix ###
    cm_table = pd.DataFrame()
    cm_table['Precision'] = con_matrix['Precision']
    cm_table['Recall']    = con_matrix['Recall']
    cm_table['Accuracy']  = con_matrix['Accuracy']
    cm_table['F1_Score']  = con_matrix['F1_Score']
    
    cm_table.index = labels
    
    # Calculate the macro-averaged for each evaluation metric
    row = pd.Series({'Precision':sum(con_matrix['Precision'])/len(con_matrix['Precision']),
                     'Recall'   :sum(con_matrix['Recall'])/len(con_matrix['Recall']),
                     'Accuracy' :sum(con_matrix['Accuracy'])/len(con_matrix['Accuracy']),
                     'F1_Score' :sum(con_matrix['F1_Score'])/len(con_matrix['F1_Score'])},
                     name='macro-averaged')
    cm_table = cm_table.append(row)
    
    # Print the confusion matrix
    print(cm_table)


In [6]:
# This function calculates the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(nb_model, attr_indexes):
    
    # Dict to store the information gain calculated of each attribute  
    igs = {}
    
    # First get all the classes
    classes = nb_model.keys()
    # Get total count of the instances
    total_count = 0
    for label in nb_model.keys():
        total_count += nb_model[label]['count']

    ### Calculate the entropy at the root node ###
    h_r = 0
    for c in classes:
        prob = nb_model[c]['count']/(total_count+0.0)
        h_r += prob*math.log(prob, 2)
    h_r *= -1
    
    # Store the entropy at the root node
    igs['h_r'] = h_r
    
    # Iterate through each attribute via its index value
    for index in attr_indexes:
        
        ### Calculate mean information for the selected attribute ###
        # First extract the count of the selected attribute (preprocessing)
        # store it in a new dict structure for easier retrieval
        p = {}
        for c in classes:
            for attr_val in nb_model[c][index].keys():
                # Build up the dict structure procedurally with attribute values 
                if attr_val not in p.keys(): p[attr_val] = {}
                p[attr_val][c] = nb_model[c][index][attr_val]
                
        # Actual calculation of mean information here
        mean_info = 0
        for attr_val in p.keys():
            # Count the number of instances that have the attribute value
            count = 0
            for label in p[attr_val].keys(): count += p[attr_val][label]
            # Calculate the entropy for the selected attribute    
            attr_entropy = 0    
            for label in p[attr_val].keys():
                prob = p[attr_val][label]/(count+0.0)
                if prob == 0: continue
                attr_entropy += prob*math.log(prob, 2)
            attr_entropy *= -1    
            mean_info += (count/total_count) * attr_entropy
        
        ### Calculate the information gain for the selected attribute ###
        ig = h_r - mean_info
        
        # Store the information gain value
        igs[index] = ig
    
    return igs


In [7]:
### Naive Bayes Classifier ###

# All the csv files 
datasets = ['anneal.csv', 
            'breast-cancer.csv',
            'car.csv',
            'cmc.csv',
            'hepatitis.csv',
            'hypothyroid.csv',
            'mushroom.csv',
            'nursery.csv',
            'primary-tumor.csv']


# Run the NB classifier on each dataset
# Evaluate the predictions for their accurary
# Calculate the information gain for each of their attributes
for dataset in datasets:
    
    print("Dataset: " + dataset +"\n") 
    
    data, target = preprocess(dataset)
    nb_model     = train(data, target)
    predictions  = predict(nb_model, data, k=0.1)
    evaluate(predictions, target)
    info_gains   = info_gain(nb_model, [i for i in range(len(data[0]))])
    
    # Print the information gains calculated
    for attr, ig in info_gains.items():
        if attr == 'h_r': 
            print("\nRoot entropy: " + str(ig))
        else :
            print("IG at %2d" %(attr) + ": " + str(ig))
    print('\n==========================================================\n')


Dataset: anneal.csv

                Precision    Recall  Accuracy  F1_Score
3                0.998418  0.922515  0.939866  0.958967
U                0.493827  1.000000  0.954343  0.661157
1                0.888889  1.000000  0.998886  0.941176
5                1.000000  1.000000  1.000000  1.000000
2                0.899083  0.989899  0.986637  0.942308
macro-averaged   0.856043  0.982483  0.975947  0.900722

Root entropy: 1.189833856204398
IG at  0: 0.40908953764451006
IG at  1: 0.0
IG at  2: 0.3060515354289405
IG at  3: 0.051344088764404106
IG at  4: 0.29108220585994726
IG at  5: 0.1471188622809556
IG at  6: 0.2137228803159087
IG at  7: 0.29223544065798446
IG at  8: 0.1261663361036096
IG at  9: 0.14107379163812883
IG at 10: 0.032488406491841815
IG at 11: 0.43517783626288575
IG at 12: 0.03870173274881061
IG at 13: 0.00043760652021185287
IG at 14: 0.03935557414283708
IG at 15: 0.021775078259213876
IG at 16: 0.037997478813511565
IG at 17: 0.03670308136440825
IG at 18: 0.0
IG at 19: 0.1

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

# Q1 Answer:
By the definition of information gain, it is “the expected reduction in the entropy (the amount of unpredictability) caused by knowing the values of an attribute”. 

If an attribute has high information gain relative to the class distribution, by knowing the value of the attribute, it reduces the amount of unpredictability for the given class, or in other words, the classifier is able to identify the class label that is generated partly from the attribute value with more confidence. Hence, attribute that have information gain increases the likelihood for the classifier to make accurate predictions. 

However, information gain alone is not sufficient to explain the behaviour of the classifier in the entropy context. Root entropy (the unpredictability in the class distribution) together with information gain can then largely suggest the effectiveness of the classifier. High root entropy suggests high unpredictability, that the class distribution is fairly even, no particular class label is significantly more probable than the other, predictions are unlikely to be accurate as a result. Hence, in most cases where root entropy is high, attributes with high information gain are necessary to make accurate predictions, conversely, in most cases where root entropy is low, it is not necessary to have attributes with high information gain relatively in order to make accurate predictions.

One particular surprising result is that the value of accuracy stays the same even with respect to different attributes. This is observed across several datasets, namely 'breast-cancer.csv', 'hepatitis.csv', 'hypothyroid.csv', and 'mushroom.csv'. The reason being that all these datasets only have two class labels, where one true positive prediction with respect to the first class label becomes the true negative prediction with respect to the second class. Since that accuracy by definition is (true positives + true negatives)/(total predictions) and that the values of sums of true positives and true negatives are the same for both class labels, they have the same accuracy value.


# Q5 Answer:
Yes, the effectiveness of the classifier is affected by the implementation of add-k smoothing. 

The issue of Naive Bayes classifier without any smoothing regimes is that it tends to discard the probability of a class label once an unseen attribute value is encountered with respect to the class label, thereby leading it to either overestimate or underestimate the probabilities of other class labels, making inaccurate predictions as a result. 

In theory, add-k smoothing could solve this issue. It adds k sets of instances to the trained model, with the amount of instances determined by the number of unique attribute values and that each instance has an unique attribute value (including the unseen attribute value). As a result, all counts are increased by k to ensure that monotonicity is maintained, and that the unseen attribute value gets a count of k as well. 

With the appropriate value of k, probability of a given class label that contains unseen attribute values could be calculated without overestimating or underestimating the likelihood of the unseen attribute values, which in turn increases the accuracy of the predictions. Hence, in most cases, it would increase the effectiveness of the classifier.

However, the value of k has to be low enough relatively to the size of the training data so that the number of instances added will not significantly change the class distribution, otherwise bias will be introduced. Moreover, the optimum value of k is rather hard to be obtained without any testing or domain knowledge in practice. As a result, without using an appropriate value of k, add-k smoothing would not increase the effectiveness of the classifier, or in worse cases, it would decrease the effectiveness instead.

In testing the effect of add-k smoothing on the classifier, with same given datasets and evaluation strategy in each test, a range of 0 (equivalent to not having a smoothing regime) to 1 was applied as the k value with total of 10 tests, starting at 0 initially then incrementally increase the k value by 0.1 until it gets to 1. The accuracy can be seen to decrease increasingly as the value of k gets larger, this observation is consistent across all datasets. One of possible explanations for this is as stated above, that the k values with range of 0 to 1 are relatively large to the size of the training data, hence artificially causes changes to the class distribution, making the predictions inaccurate as a result.

